# Primeira entrega

Planejamento da Solução
 - Ler o artigo para pensar na solução

Coleta de dados
 - Buscar uma forma de conectar com o banco de dados da AWS

# 0.0 Imports

In [1]:
import pickle

import numpy    as np
import pandas   as pd
import seaborn  as sns

from scipy              import stats
from IPython.display    import Image

import funcoes_suporte as fs

## 0.1 Help Functions

In [2]:
fs.jupyter_settings(altura= 18, largura= 14, fonte= 11)
fs.supressao_notacao()

## 0.2 Load interim data

In [3]:
df = pickle.load(open('../data/interim/x_treino.pkl', 'rb'))
df.head(10)

,id,genero,idade,codigo_regiao,contato_cliente,cnh,idade_veiculo,veiculo_danificado,seguro_previo_automovel,premio_anual,cliente_dias_contrato
55914,55915,Male,33,29.00,124.00,1,1-2 Year,Yes,0,26761.00,139
179876,179877,Male,55,28.00,26.00,1,> 2 Years,Yes,0,27875.00,285
300095,300096,Female,26,28.00,152.00,1,< 1 Year,No,1,43562.00,118
60078,60079,Male,57,38.00,124.00,1,1-2 Year,No,1,51365.00,249
312426,312427,Male,27,42.00,152.00,1,< 1 Year,Yes,0,26590.00,161
62215,62216,Female,30,41.00,152.00,1,< 1 Year,No,1,26252.00,12
239830,239831,Female,24,36.00,152.00,1,< 1 Year,No,1,30873.00,130
264049,264050,Male,71,29.00,26.00,1,> 2 Years,Yes,0,33122.00,104
198247,198248,Male,50,28.00,26.00,1,1-2 Year,No,1,34822.00,197
52010,52011,Female,24,36.00,152.00,1,< 1 Year,No,1,2630.00,18


# 1.0 Hipoteses 
Iniciada no __2º Cíclo CRISP__

## 1.1 Mind map

### 1.2.1 Cliente

__2º Cíclo CRISP__

1. Clientes entre 35 e 65 anos responderam em média mais 'sim' no questionário.

2. Clientes do gênero masculinos tem uma maior probabilidade de responder 'não' no questionário.

3. Clientes de da mesma região não divergiram a resposta em mais de 20% (Pareto).

4. Clientes que não possuem CNH responderam em média mais 'sim' no questionário.

### 1.2.2 Veículo

__2º Cíclo CRISP__

1. Clientes com veículos Não danificados responderam em média mais 'não' no questionário.

2. Clientes com veículos acima de 2 anos de idade responderam em média mais 'não' no questionário.

3. Clientes com veículos até 1 ano de idade e com seguro automotivo e responderam em média mais 'sim' no questionário.

### 1.2.3 Seguro

__2º Cíclo CRISP__

1. Clientes que já possuem seguro automotívo resmponderam em média mais 'sim' no questionário.

2. Clientes com até 90 dias de seguro responderam em média mais 'não' no questionário.

3. 30% Clientes dos prêmios mais baixos responderam em média mais 'não' no questionário.

# 2.0 Feature Engenniring

## 1.1 Fetures necessárias

__Features Reestruturadas__ 1º Ciclo CRISP

1. idade_veiculo - transformar de str para int (1, 2, 3)

2. veiculo_danificado - transformar de str para int (0, 1)

__Features Derivadas__ 2º Ciclo CRISP

1. Idade - Jovem (x = min, x <= 35), Adulto (x > 35 - x <= 65), Idoso (x > 65).

2. Idade Veículo - Pouca Manutenção (x = min, x <= 1), Médio Manutenção (x > 1 - x <= 2), Muita Manutenção (x > 2).

3. Cliente dias Contrato - semana 7 dias completos; mês 30 dias completos.

3º Ciclo CRIPS

1. Prêmio anual - Agrupar o premio anual em 5 classificações x < 2 .... x > 10 (2.5,5,7.5,10,+)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266776 entries, 55914 to 121958
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       266776 non-null  int64  
 1   genero                   266776 non-null  object 
 2   idade                    266776 non-null  int64  
 3   codigo_regiao            266776 non-null  float64
 4   contato_cliente          266776 non-null  float64
 5   cnh                      266776 non-null  int64  
 6   idade_veiculo            266776 non-null  object 
 7   veiculo_danificado       266776 non-null  object 
 8   seguro_previo_automovel  266776 non-null  int64  
 9   premio_anual             266776 non-null  float64
 10  cliente_dias_contrato    266776 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 24.4+ MB


In [5]:
df['idade_veiculo'] = df['idade_veiculo'].apply(lambda x: 1 if x == '< 1 Year' else
                                                          2 if x == '1-2 Year' else 3)

In [6]:
df['veiculo_danificado'] = df['veiculo_danificado'].apply(lambda x: 1 if x == 'Yes' else 0)

In [7]:
df['idade_class_etaria'] = df['idade'].apply(lambda x: 1 if x <= 35 else
                                                       2 if x > 35 & x <= 65 else 3)

In [8]:
bins = [0, 25000, 50000, 75000, 100000, float('inf')]
label = [1, 2, 3, 4, 5]

df['premio_anual_cat'] = pd.cut(x = df['premio_anual'], bins = bins, labels = label, right = False).astype('int64')

In [9]:
df['semanas_contrato'] = df['cliente_dias_contrato']/7

In [10]:
df['meses_contrato'] = df['cliente_dias_contrato']/30

In [11]:
df.sample(15)

,id,genero,idade,codigo_regiao,contato_cliente,cnh,idade_veiculo,veiculo_danificado,seguro_previo_automovel,premio_anual,cliente_dias_contrato,idade_class_etaria,premio_anual_cat,semanas_contrato,meses_contrato
290618,290619,Male,75,3.00,26.00,1,2,0,0,37126.00,141,2,2,20.14,4.70
61782,61783,Male,25,37.00,152.00,1,1,1,0,26130.00,277,1,2,39.57,9.23
147668,147669,Male,25,29.00,152.00,1,1,0,1,26691.00,126,1,2,18.00,4.20
191517,191518,Male,37,8.00,124.00,1,2,0,1,46776.00,139,2,2,19.86,4.63
180721,180722,Female,24,29.00,152.00,1,1,0,1,22696.00,286,1,1,40.86,9.53
10936,10937,Male,22,46.00,152.00,1,1,0,1,35595.00,172,1,2,24.57,5.73
301483,301484,Male,22,33.00,152.00,1,1,0,1,26730.00,223,1,2,31.86,7.43
240378,240379,Male,44,8.00,124.00,1,2,1,0,45057.00,101,2,2,14.43,3.37
23844,23845,Female,32,28.00,124.00,1,1,1,0,26850.00,160,1,2,22.86,5.33
209975,209976,Female,41,28.00,26.00,1,2,0,0,2630.00,206,2,1,29.43,6.87


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266776 entries, 55914 to 121958
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       266776 non-null  int64  
 1   genero                   266776 non-null  object 
 2   idade                    266776 non-null  int64  
 3   codigo_regiao            266776 non-null  float64
 4   contato_cliente          266776 non-null  float64
 5   cnh                      266776 non-null  int64  
 6   idade_veiculo            266776 non-null  int64  
 7   veiculo_danificado       266776 non-null  int64  
 8   seguro_previo_automovel  266776 non-null  int64  
 9   premio_anual             266776 non-null  float64
 10  cliente_dias_contrato    266776 non-null  int64  
 11  idade_class_etaria       266776 non-null  int64  
 12  premio_anual_cat         266776 non-null  int64  
 13  semanas_contrato         266776 non-null  float64
 14  meses

In [13]:

def _fe_idade_veiculo(df):
    """Codifica idade_veiculo em faixas discretas"""
    df['idade_veiculo'] = df['idade_veiculo'].apply(
        lambda x: 1 if x == '< 1 Year' else
                  2 if x == '1-2 Year' else
                  3
    )
    return df


def _fe_veiculo_danificado(df):
    """Transforma variável binária de texto ('Yes'/'No') em 1/0"""
    df['veiculo_danificado'] = df['veiculo_danificado'].apply(
        lambda x: 1 if x == 'Yes' else 0
    )
    return df


def _fe_idade_class_etaria(df):
    """Cria faixa etária com base em idade"""
    df['idade_class_etaria'] = df['idade'].apply(
        lambda x: 1 if x <= 35 else
                  2 if (x > 35) and (x <= 65) else
                  3
    )
    return df


def _fe_premio_anual(df):
    """Cria categoria de prêmio anual com base em faixas de valores"""
    bins = [0, 25000, 50000, 75000, 100000, float('inf')]
    labels = [1, 2, 3, 4, 5]
    df['premio_anual_cat'] = pd.cut(
        x=df['premio_anual'],
        bins=bins,
        labels=labels,
        right=False
    )
    float(df['premio_anual_cat']).astype('int64')
    return df

def _fe_semanas_contrato(df):
    df['semanas_contrato'] = df['cliente_dias_contrato']/7
    return df

def _fe_meses_contrato(df):
    df['meses_contrato'] = df['cliente_dias_contrato']/30
    return df

def _fe_contrato(df):
    df = _fe_semanas_contrato(df)
    df = _fe_meses_contrato(df)
    return df

DICTFE = {
    'idade_veiculo': _fe_idade_veiculo,
    'veiculo_danificado': _fe_veiculo_danificado,
    'idade': _fe_idade_class_etaria,
    'premio_anual': _fe_premio_anual,
    'cliente_dias_contrato': _fe_contrato,
}

def fe(df):
    for col, fn in DICTFE.items():
        if col in df.columns:
            df = fn(df)
        else:
            print(f"[Aviso] Coluna {col} ausente, pulando.")
    return df

# 3.0 Filter Data

Sem filtro __1º Ciclo CRISP__

# 4.0 Export Pickle interim Data

In [14]:
pickle.dump(df,open('../data/interim/x_treino_fe.pkl','wb'))

In [15]:
del df